In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

cols = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter'.split()
colt = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter number'.split()

name = 'LH'

### 1ST Batch

In [2]:
div_year = 2018
div_quarter = 2
ids = '553'

In [3]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,LH,30000.0,345764.14,2018-08-23


In [4]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,228,LH,2020,4,"42,000",0.3000,"11,340.00",2021-05-06,2021-05-21
1,17,LH,2020,2,"45,000",0.2000,"8,100.00",2020-08-27,2020-09-11
2,9,LH,2019,4,"45,000",0.4000,"16,200.00",2020-05-07,2020-05-22
3,44,LH,2019,2,"45,000",0.3000,"12,150.00",2019-08-28,2019-09-11
4,61,LH,2018,4,"45,000",0.3500,"14,175.00",2019-05-03,2019-05-21
5,92,LH,2018,2,"30,000",0.4000,"10,800.00",2018-08-28,2018-09-11


In [5]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,LH,30000.0,345764.14,0.30,9000.0,2018-08-23,2021-05-06,2021-05-21,1002,0.95,2020,4
1,LH,30000.0,345764.14,0.20,6000.0,2018-08-23,2020-08-27,2020-09-11,750,0.84,2020,2
2,LH,30000.0,345764.14,0.40,12000.0,2018-08-23,2020-05-07,2020-05-22,638,1.99,2019,4
3,LH,30000.0,345764.14,0.30,9000.0,2018-08-23,2019-08-28,2019-09-11,384,2.47,2019,2
4,LH,30000.0,345764.14,0.35,10500.0,2018-08-23,2019-05-03,2019-05-21,271,4.09,2018,4
5,LH,30000.0,345764.14,0.40,12000.0,2018-08-23,2018-08-28,2018-09-11,19,66.67,2018,2


In [6]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div1 = df_grp
div1

name
LH    58500.0
Name: div_amt, dtype: float64

In [7]:
file_name = 'div-' + name + '.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/div-LH.csv',
 '\\Users\\User\\iCloudDrive\\div-LH.csv',
 '\\Users\\User\\Dropbox\\div-LH.csv')

In [8]:
df_merge[colt].to_csv(output_file)
df_merge[colt].to_csv(data_file)
df_merge[colt].to_csv(box_file)

### Second batch of purchase

In [9]:
div_year = 2018
div_quarter = 4
ids = '586,602'

In [10]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,LH,12000.0,123573.09,2018-11-16


In [11]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,228,LH,2020,4,"42,000",0.3000,"11,340.00",2021-05-06,2021-05-21
1,17,LH,2020,2,"45,000",0.2000,"8,100.00",2020-08-27,2020-09-11
2,9,LH,2019,4,"45,000",0.4000,"16,200.00",2020-05-07,2020-05-22
3,44,LH,2019,2,"45,000",0.3000,"12,150.00",2019-08-28,2019-09-11
4,61,LH,2018,4,"45,000",0.3500,"14,175.00",2019-05-03,2019-05-21


In [12]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,LH,12000.0,123573.09,0.30,3600.0,2018-11-16,2021-05-06,2021-05-21,917,1.16,2020,4
1,LH,12000.0,123573.09,0.20,2400.0,2018-11-16,2020-08-27,2020-09-11,665,1.07,2020,2
2,LH,12000.0,123573.09,0.40,4800.0,2018-11-16,2020-05-07,2020-05-22,553,2.56,2019,4
3,LH,12000.0,123573.09,0.30,3600.0,2018-11-16,2019-08-28,2019-09-11,299,3.56,2019,2
4,LH,12000.0,123573.09,0.35,4200.0,2018-11-16,2019-05-03,2019-05-21,186,6.67,2018,4


In [13]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div2 = df_grp
div2

name
LH    18600.0
Name: div_amt, dtype: float64

In [14]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### End of process

In [15]:
div_year = 2018
div_quarter = 2

In [16]:
sql = """
SELECT id, name, year, quarter, number, ppu, number * ppu AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year, quarter
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,92,LH,2018,2,"30,000",0.4000,"12,000.00",2018-08-28,2018-09-11
1,61,LH,2018,4,"45,000",0.3500,"15,750.00",2019-05-03,2019-05-21
2,44,LH,2019,2,"45,000",0.3000,"13,500.00",2019-08-28,2019-09-11
3,9,LH,2019,4,"45,000",0.4000,"18,000.00",2020-05-07,2020-05-22
4,17,LH,2020,2,"45,000",0.2000,"9,000.00",2020-08-27,2020-09-11
5,228,LH,2020,4,"42,000",0.3000,"12,600.00",2021-05-06,2021-05-21


In [17]:
dividends.div_amt.sum()

80850.0